In [3]:
import re
import requests 
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

numbeo data by countries

In [5]:
def is_float(element) -> bool:
    try:
        float(element)
        return True
    except ValueError:
        return False

In [13]:
################# test cell ##################
cityURL = "https://www.numbeo.com/traffic/city_result.jsp?country=Russia&city=Moscow&displayCurrency=USD"

res = requests.get(cityURL).text
soup = BeautifulSoup(res, "html.parser")
table = soup.find("table", class_ = "table_indices")

for row in table.find_all("tr"):
    column = row.find_all("td")
    if column != []:
        name = column[0].text.strip()
        #price = float(re.findall(r"[-+]?(?:\d*\.\d+|\d+)", column[2].text.strip())[0])
        price = column[1].text.strip().replace(',',"")
        
        print(name, price)

Traffic Index: 215.55
Time Index (in minutes): 50.58
Time Exp. Index: 6762.48
Inefficiency Index: 222.75
CO2 Emission Index: 4599.17


In [17]:
class webScraping:

    def __init__(self, url_target) -> None:
        self.DataDict = dict()
        self.DataDict["CountryName"] = list()
        self.DataDict["CityName"] = list()
        self.url_target = url_target


    def getCountyNameList(self, countryURL):

        res = requests.get(countryURL).text
        soup = BeautifulSoup(res, "html.parser")
        countyrlist = list()
        
        for a in  soup.find_all('a',href = True):
            if 'country_result' in a['href']:
                countyrlist.append(a["href"].split("=")[1])
        return countyrlist

    def getCountriesCostOfLiving(self, cityURL, countryName, cityName):

        res = requests.get(cityURL).text
        soup = BeautifulSoup(res, "html.parser")

        if self.url_target == 'traffic' or self.url_target == 'quality-of-life':
            table = soup.find("table", class_ = "table_indices")
        else:
            table = soup.find("table", class_ = "data_wide_table")

        if table == None:
            return

        self.DataDict["CountryName"].append(countryName)
        self.DataDict["CityName"].append(cityName)

        for row in table.find_all("tr"):
            column = row.find_all("td")
            if column != []:
                name = column[0].text.strip()
                
                if self.url_target == 'crime' or self.url_target == 'health-care' or self.url_target == 'pollution':
                    price = float(re.findall(r"[-+]?(?:\d*\.\d+|\d+)", column[2].text.strip())[0])
                elif self.url_target == 'traffic':
                    price = column[1].text.strip().replace(',',"")
                else:
                    price = column[1].text.strip()
                
                if name not in [i for i in self.DataDict.keys()]:
                    self.DataDict[name] = list()
                    self.DataDict[name].append(price)
                else:
                    self.DataDict[name].append(price)


    def CountryNameOperation(self, countryName): 

        countryName = countryName.replace("%28", "(")
        countryName = countryName.replace("%29", ")")
        countryName = countryName.replace("+", " ")
        
        return countryName


    def dataMerge(self):
        
        countyrsList = self.getCountyNameList(countryURL="https://www.numbeo.com/" + self.url_target + "/")
        print("Country names successfully imported (total number of countries : {})\n".format(len(countyrsList)))
        counter = 1
        for countryName in tqdm(countyrsList):
            countryName_ = self.CountryNameOperation(countryName)

            countryURL = "https://www.numbeo.com/" + self.url_target + "/country_result.jsp?country=" + countryName_
            res = requests.get(countryURL).text
            soup = BeautifulSoup(res, "html.parser")
            citylist = list()
            for a in  soup.find_all('option', value = True):
                city_name = a["value"].split("=")[0]
                if len(city_name) > 3:
                    citylist.append(city_name)

            for city in citylist:

                url = "https://www.numbeo.com/" + self.url_target + "/city_result.jsp?country=" + countryName_ + "&city=" + city + "&displayCurrency=USD"
                self.getCountriesCostOfLiving(url, countryName_, city)
                counter += 1

        return self.DataDict

    def getDataFrame(self):
        dataDict = self.DataDict
        counterDict = dict()
        
        for i in dataDict:
            if str(len(dataDict[i])) not in [ky for ky in counterDict.keys()]:
                counterDict[str(len(dataDict[i]))] = 1
            else:
                counterDict[str(len(dataDict[i]))] += 1
                
        maxValue = max(counterDict, key = counterDict.get)
        lastDict = dict()

        for i in dataDict:
            if len(dataDict[i]) == int(maxValue):
                lastDict[i] = dataDict[i]
        
        return pd.DataFrame(lastDict)
            

scrap_targets = {"cost": "cost-of-living",
                 "property": "property-investment",
                 "quality": "quality-of-life",
                 "crime": "crime",
                 "health": "health-care",
                 "pollution": "pollution",
                 "traffic": "traffic"}


WS = webScraping(scrap_targets["quality"])

WS.dataMerge()

data = WS.getDataFrame()

Country names successfully imported (total number of countries : 233)



100%|██████████| 233/233 [1:17:17<00:00, 19.90s/it]


In [18]:
data.to_csv('numbeo_quality.csv')
data

,CountryName,CityName,Quality of Life Index:
0,Afghanistan,Kabul,?
1,Aland Islands,Inter,?
2,Aland Islands,Mariehamn,200.18
3,Albania,Durres,97.14
4,Albania,Elbasan,?
...,...,...,...
2384,Yemen,Aden,?
2385,Yemen,Sana (Sanaa),?
2386,Yemen,Sanaa,?
2387,Zambia,Lusaka,?
